# Emotion Chat Bot

In [1]:
from operator import itemgetter

import torch
from libs import (
    EmotionModel,
    ResponseGeneratorPipeline,
    SimilarityAnalyser,
    generate_dummy_representation,
    get_sentiment_composition,
)
from libs.FullModel import (
    create_candidates_buffer,
    get_top_emotion,
)
from sympy.core.random import randint
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)
from transformers.utils.import_utils import candidates
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Load Each Module

### Response Generator

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
    attn_implementation="flash_attention_2",
    pretraining_tp=1,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '


In [3]:
response_generator = ResponseGeneratorPipeline(
    model,
    tokenizer,
    framework="pt",
    task="conversation-generation",
    num_workers=16,
    torch_dtype="auto",
    add_special_tokens=True,
    truncation=False,
    padding=True,
)

In [4]:
FastLanguageModel.for_inference(response_generator.model)
response_generator.model = torch.compile(
    response_generator.model, mode="reduce-overhead"
)

### Sentiment Analyzer

In [5]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [6]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [7]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=sentiment_analysis_model,
    tokenizer=sentiment_analysis_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [8]:
sentiment_analyzer.model = torch.compile(
    sentiment_analyzer.model, mode="reduce-overhead"
)

In [9]:
print(sentiment_analyzer.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Predictor

In [10]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [11]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [12]:
emotion_predictor = pipeline(
    "sentiment-analysis",
    model=emotion_predictor_model,
    tokenizer=emotion_predictor_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [13]:
emotion_predictor.model = torch.compile(emotion_predictor.model, mode="reduce-overhead")

In [14]:
print(emotion_predictor.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Model

In [15]:
emotion_model = EmotionModel.from_pretrained(
    "hermeschen1116/emotion_model_for_emotion_chat_bot"
)

In [16]:
emotion_model = torch.compile(emotion_model, mode="reduce-overhead")

In [17]:
print(emotion_model)

OptimizedModule(
  (_orig_mod): EmotionModel(
    (_EmotionModel__weight_Q): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__weight_K): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__dropout): Dropout(p=0.5, inplace=False)
    (_EmotionModel__weight_D): Linear(in_features=7, out_features=7, bias=True)
  )
)


### Similarity Analyzer

In [18]:
threshold: float = 0.5

In [19]:
similarity_analyzer = SimilarityAnalyser(threshold)

## Combine All Modules

### Initialize

In [20]:
chat_buffer_size: int = 10

In [21]:
streamer = TextStreamer(
    tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [22]:
ideal_bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
ideal_bot_emotion_representation

tensor([0.8282, 0.6523, 0.6510, 0.6189, 0.0338, 0.0171, 0.3525])

In [23]:
bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
bot_emotion_representation

tensor([0.8252, 0.0665, 0.8577, 0.1676, 0.1080, 0.2504, 0.3674])

In [47]:
system_prompt: str = input("Enter your system prompt: ").strip()
bot_message: str = "Talk to me..."

In [48]:
chat_buffer: list = [
    {"role": "system", "content": {"emotion": "", "dialog": system_prompt}}
]

In [49]:
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}}]

In [50]:
while True:
    user_response: str = input(bot_message).strip()
    if user_response == "quit":
        break

    user_emotion: list = sentiment_analyzer(user_response)
    chat_buffer.append(
        {
            "role": "user",
            "content": {
                "emotion": get_top_emotion(user_emotion),
                "dialog": user_response,
            },
        }
    )

    user_emotion_composition: Tensor = get_sentiment_composition(user_emotion)

    bot_emotion_representation = emotion_model.forward(
        user_emotion_composition, bot_emotion_representation
    )

    bot_chat_simulations: list = create_candidates_buffer(chat_buffer)
    bot_chat_simulations = [
        chat[0] for chat in response_generator(bot_chat_simulations, streamer=streamer)
    ]
    print()
    # bot_chat_simulations = [chat[0] for chat in response_generator(bot_chat_simulations)]
    bot_chat_simulations = list(
        filter(lambda chat: chat[-1]["content"]["dialog"] != "", bot_chat_simulations)
    )

    bot_response_simulations: list = [
        chat[-1]["content"] for chat in bot_chat_simulations
    ]

    user_future_emotion_composition_simulations: dict = {
        response["emotion"]: get_sentiment_composition(
            emotion_predictor(response["dialog"])
        )
        for response in bot_response_simulations
    }

    bot_future_emotion_representations: dict = {
        k: emotion_model.forward(v, bot_emotion_representation)
        for k, v in user_future_emotion_composition_simulations.items()
    }

    emotion_representation_similarity_scores: list = similarity_analyzer(
        list(bot_future_emotion_representations.values()),
        ideal_bot_emotion_representation,
    ).tolist()

    emotions: list = [
        "neutral",
        "anger",
        "disgust",
        "fear",
        "happiness",
        "sadness",
        "surprise",
    ]
    bot_best_response_emotion: str = emotions[
        max(enumerate(emotion_representation_similarity_scores), key=itemgetter(1))[0]
    ]

    if len(chat_buffer) == chat_buffer_size + 1:
        chat_buffer.pop(1)
    chat_buffer.append(
        {"role": "bot", "content": {"emotion": bot_best_response_emotion, "dialog": ""}}
    )

    chat_buffer = response_generator(chat_buffer, streamer=streamer)[0]
    print()
    if chat_buffer[-1]["content"]["dialog"] == "":
        chat_buffer[-1]["content"]["dialog"] = dict(
            filter(lambda x: x.key() == bot_best_response_emotion, bot_chat_simulations)
        ).value()
    bot_message = chat_buffer[-1]["content"]["dialog"]

   neutral   Hello, how are you?     neutral    Fine, thank you. 
   neutral   Hello, how are you?     anger   
   neutral   Hello, how are you?     disgust   
   neutral   Hello, how are you?     fear    Hi! I'm terribly nervous. 
   neutral   Hello, how are you?     happiness     Fine, thanks. 
   neutral   Hello, how are you?     sadness   
   neutral   Hello, how are you?     surprise     Hi! I haven't seen you for ages! 

   neutral   Hello, how are you?     anger    Hi! I'm fine! 

   neutral   Hello, how are you?     anger   Hi! I'm fine!  
    happiness   Good, nice to meet you.     neutral    Nice to meet you too. 
   neutral   Hello, how are you?     anger   Hi! I'm fine!  
    happiness   Good, nice to meet you.     anger   
   neutral   Hello, how are you?     anger   Hi! I'm fine!  
    happiness   Good, nice to meet you.     disgust   
   neutral   Hello, how are you?     anger   Hi! I'm fine!  
    happiness   Good, nice to meet you.     fear   
   neutral   Hello, how a

AttributeError: 'list' object has no attribute 'key'